In [1]:
# Imports
import matplotlib.pyplot as plt
import cv2
import torch
import pathlib
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import logging
import shutil
import json

# Constants 
CWD = pathlib.Path(os.path.abspath(""))
GIT_ROOT = CWD.parent.parent
DATA_DIR = GIT_ROOT / "data" / 'ICMI2024'
REID_DB = DATA_DIR / 'reid' / 'db'
OUTPUT_DIR = DATA_DIR / 'final'

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [33]:
def final_tracking(tracking_file: pathlib.Path, gaze_file: pathlib.Path, reid_file: pathlib.Path, output_file: pathlib.Path):
    assert tracking_file.exists(), f"Tracking file {tracking_file} does not exist"
    assert gaze_file.exists(), f"Gaze file {gaze_file} does not exist"
    assert reid_file.exists(), f"Reid file {reid_file} does not exist"

    # Load the tracking file
    tracking = pd.read_csv(tracking_file)
    gaze = pd.read_csv(gaze_file)
    
    with open(reid_file, 'r') as f:
        reid = json.load(f)

    # Resort the tracking file
    # tracking = tracking.sort_values(by=['Frame', 'Face_Index'])

    # Create container for the output
    tracking['reid'] = None
    tracking['gaze'] = None

    # Get the total number of frames (max frame number)
    max_frame = tracking['Frame'].max()
    # max_frame = 300

    # Iterate over the tracking
    for i in tqdm(range(max_frame), total=max_frame):

        # Get the faces and gaze for the current frame
        frame_faces = tracking[tracking['Frame'] == i]
        frame_gaze = gaze[gaze['frame'] == i]

        # Iterate over the faces
        for j, face in frame_faces.iterrows():
            
            # Get tracking ID
            tracking_id = int(face['Student_ID'])

            # Convert the tracking_id to REID ID
            if str(tracking_id) in reid:
                reid_id = reid[str(tracking_id)]
            else:
                reid_id = None
                # print(f"Tracking ID {tracking_id} not found in REID database")

            # Get the gaze for the current face
            try:
                gaze_data = frame_gaze[frame_gaze['src_tracked_id'] == tracking_id].iloc[0]
            except IndexError:
                gaze_data = {'dst_tracked_id': None}
                # print(f"Tracking ID {tracking_id} not found in gaze database")

            # Set the data
            tracking.at[j, 'reid'] = reid_id
            if not pd.isna(gaze_data['dst_tracked_id']) or gaze_data['dst_tracked_id'] is not None:
                if str(gaze_data['dst_tracked_id']) in ['display', 'floor']:
                    tracking.at[j, 'gaze'] = str(gaze_data['dst_tracked_id'])
                elif str(gaze_data['dst_tracked_id']) in reid:
                    if reid_id != reid[str(gaze_data['dst_tracked_id'])]:
                        tracking.at[j, 'gaze'] = reid[str(gaze_data['dst_tracked_id'])]
                        assert reid_id != reid[str(gaze_data['dst_tracked_id'])], f"Reid ID {reid_id} is the same as gaze ID {reid[str(gaze_data['dst_tracked_id'])]}"

    # Save the output
    tracking.to_csv(output_file, index=False)

# final_tracking(
#     DATA_DIR / 'trackings' / 'Day1Group1Camera2_with_student_IDs.csv',
#     DATA_DIR / 'gaze_vectors' / 'gaze_vector_d1g1_raytraced.csv',
#     DATA_DIR / 'reid' / 'tables' / 'd1g1-cam2_cleaned.json',
#     OUTPUT_DIR / 'video_sampled_d1g1.csv'
# )

# final_tracking(
#     DATA_DIR / 'trackings' / 'Day1Group2Camera2_with_student_IDs.csv',
#     DATA_DIR / 'gaze_vectors' / 'gaze_vector_d1g2_raytraced.csv',
#     DATA_DIR / 'reid' / 'tables' / 'd1g2-cam2_cleaned.json',
#     OUTPUT_DIR / 'video_sampled_d1g2.csv'
# )

# final_tracking(
#     DATA_DIR / 'trackings' / 'Day2Group1Camera2_with_student_IDs.csv',
#     DATA_DIR / 'gaze_vectors' / 'gaze_vector_d2g1_raytraced.csv',
#     DATA_DIR / 'reid' / 'tables' / 'd2g1-cam2_cleaned.json',
#     OUTPUT_DIR / 'video_sampled_d2g1.csv'
# )

# final_tracking(
#     DATA_DIR / 'trackings' / 'Day2Group2Camera2_with_student_IDs.csv',
#     DATA_DIR / 'gaze_vectors' / 'gaze_vector_d2g2_raytraced.csv',
#     DATA_DIR / 'reid' / 'tables' / 'd2g2-cam2_cleaned.json',
#     OUTPUT_DIR / 'video_sampled_d2g2.csv'
# )

100%|██████████| 18833/18833 [00:44<00:00, 422.11it/s]


In [3]:
# Create 5 second windows for the data

MAPPPING = {
    "s1": "rose",
    "s2": "DaPaw",
    "s3": "SJ 3747",
    "s4": "Taylow Swift",
    "s5": "EL",
    "s6": "Messi",
    "s7": "S7",
    "teacher": "teacher",
    "r1": "r1",
    "r2": "r2",
    'display': 'display',
    'floor': 'floor',
    None: None
}

TIME_WINDOW = 2 # seconds
FPS = 30

def process_window(df: pd.DataFrame) -> dict:
    
    # Group by reid
    groups = df.groupby('reid')

    # Compute the most frequent gaze value
    gaze = groups['gaze'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None)

    # Create a dictionary for the output, where the key is the reid and the value is the gaze
    output = {k: None for k in MAPPPING.values() if k not in ['display', 'floor', None]}
    output
    for reid, gaze in gaze.items():
        output[MAPPPING[reid]] = MAPPPING[gaze]
    
    return output

def time_window_dataset(final_file, output_file):
    assert final_file.exists(), f"Final file {final_file} does not exist"

    # Load the final file
    final = pd.read_csv(final_file)
    output_container = {}

    # Compute timestamps given frame number
    final['timestamp'] = final['Frame'] / FPS

    # Split the data into time windows
    time_windows = []
    for i in range(int(final['timestamp'].max() / TIME_WINDOW)):
        start = i * TIME_WINDOW
        end = (i + 1) * TIME_WINDOW
        time_windows.append((start, end))

    # Create the time windows
    for i, (start, end) in enumerate(time_windows):
        window = final[(final['timestamp'] >= start) & (final['timestamp'] < end)]

        # Process the window
        output = process_window(window)

        # Add value for the timestamp
        output['timestamp'] = (i+1) * TIME_WINDOW

        for key, value in output.items():
            if key not in output_container:
                output_container[key] = []
            output_container[key].append(value)

    # Save the output
    df = pd.DataFrame(output_container)

    # Make the timestamp the first column
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    df.to_csv(output_file, index=False)

time_window_dataset(
    OUTPUT_DIR / 'video_sampled_d1g1.csv',
    OUTPUT_DIR / f'd1g1_time_window_{TIME_WINDOW}.csv',
)
time_window_dataset(
    OUTPUT_DIR / 'video_sampled_d1g2.csv',
    OUTPUT_DIR / f'd1g2_time_window_{TIME_WINDOW}.csv',
)
time_window_dataset(
    OUTPUT_DIR / 'video_sampled_d2g1.csv',
    OUTPUT_DIR / f'd2g1_time_window_{TIME_WINDOW}.csv',
)
time_window_dataset(
    OUTPUT_DIR / 'video_sampled_d2g2.csv',
    OUTPUT_DIR / f'd2g2_time_window_{TIME_WINDOW}.csv',
)